# Top 100 UK YouTubers - Excel to SQL to Power BI Project

[View on GitHub](https://github.com/Naol-Legesse/Portfolio-Projects/tree/main/Top%20100%20UK%20YouTubers)

## Table Of Contents:
- [Objectives](#Objectives)
- [Data source](#Datasource)
- [Tools used](#Toolsused)
- [Development process](#Development)
- [Visualization](#Visualization)
- [Dax measures used in Power BI](#Daxmeasures)
- [Analysis](#Analysis)
- [Discovery](#Discovery)
- [Recommendation](#Recommendation)

### Objectives <a class="anchor" id="Objectives"></a>

The marketing team wants to find out who the top UK YouTubers are in 2024 to decide with whom it would be best to run marketing campaigns throughout the rest of the year.
The idea is to create a dashboard that provides insights into the top UK YouTubers in 2024 that includes their:

    subscriber count
    total views
    total videos, and
    engagement metrics
<br>

### Data source <a class="anchor" id="Datasource"></a>

The analyst needs data on the top UK YouTubers in 2024, and this could be sourced from Kaggle (an Excel extract), [see here to find it](https://www.kaggle.com/datasets/bhavyadhingra00020/top-100-social-media-influencers-2024-countrywise?resource=download)

In [75]:
%%html
<style>
  table {margin-left: 0 !important;}
</style>

### Tools used <a class="anchor" id="Toolsused"></a>

| Tool | Purpose |
| --- | ---|
| Excel | Exploring the data |
| SQL Server | Cleaning, testing, and analyzing the data |
| Power BI | Visualizing the data via interactive dashboards |
| GitHub | Hosting the project documentation and version control |
<br>

### Development process <a class="anchor" id="Development"></a>

The general approach in creating this solution from start to finish is as follows:

    1. Get the data from Kaggle
    2. Explore the data in Excel
    3. Load the data into SQL Server
    4. Clean the data with SQL
    5. Test the data with SQL
    6. Visualize the data in Power BI
    7. Generate the findings based on the insights

Initial observations of the dataset with Excel:

    1. There are at least 4 columns that contain the data we need for this analysis.
    2. The first column contains the channel name with corresponding channel IDS, which are separated by a @ symbol - we need to extract
       the channel names from this.
    3. Some of the cells and header names are in a different language - we need to confirm if these columns are needed, and if so, we need
       to address them.
    4. We have more data than we need, so some of these columns would need to be removed

Dataset exploration with SQL Server:

    1. Remove unnecessary columns by only selecting the ones you need
``` sql
Select NOMBRE, total_subscribers, total_views, total_videos
from Top_100_UK_youtube_channels
```
    2. Extract Youtube channel names from the first column, which contains the channel name with channel IDS, separated by the "@" symbol
```sql
-- use charindex() to find the location of @
select CHARINDEX('@',NOMBRE,1) from Top_100_UK_youtube_channels

-- use substring() to extract the channel name
select SUBSTRING(NOMBRE,1,CHARINDEX('@',NOMBRE,1)-2) from Top_100_UK_youtube_channels

-- use CAST() to change to string
Select CAST(SUBSTRING(NOMBRE,1,CHARINDEX('@',NOMBRE,1)-2) as varchar(100)) as Channel_Name,
total_subscribers,
total_videos,
total_views
from Top_100_UK_youtube_channels

-- Create a view for next step 
CREATE VIEW view_1_Top_100_UK_youtube_channels AS
	Select CAST(SUBSTRING(NOMBRE,1,CHARINDEX('@',NOMBRE,1)-2) as varchar(100)) as Channel_Name,
		total_subscribers,
		total_videos,
		total_views
	from Top_100_UK_youtube_channels
```
    3. Test the dataset
```sql
	/* guidelines to test out the dataset

	There should be 100 entries (row count)
	There should be 4 fields (column count)
	Channel name must be string ; while total_subscribers, total_videos, total_views must be integers (data type check)
	There must be no duplicate entries

	*/

	-- row count test
	Select COUNT(*) AS row_count 
	FROM view_1_Top_100_UK_youtube_channels

	-- column count test
	SELECT * 
	FROM INFORMATION_SCHEMA.COLUMNS
	WHERE TABLE_NAME = 'view_1_Top_100_UK_youtube_channels'

	OR

	SELECT COUNT(*) AS column_count 
	FROM INFORMATION_SCHEMA.COLUMNS
	WHERE TABLE_NAME = 'view_1_Top_100_UK_youtube_channels'

	-- data type test
	SELECT COLUMN_NAME, DATA_TYPE 
	FROM INFORMATION_SCHEMA.COLUMNS
	WHERE TABLE_NAME = 'view_1_Top_100_UK_youtube_channels'

	-- duplicate test
	SELECT Channel_Name, COUNT(Channel_Name) 
	FROM view_1_Top_100_UK_youtube_channels
	GROUP BY Channel_Name
	HAVING COUNT(Channel_Name) >1
```
    4. Load dataset to Power BI
<br>

### Visualization <a class="anchor" id="Visualization"></a>

![title](Top100UKyoutubersdashboardsnippet.gif)
<br>

### Dax measures used in Power BI <a class="anchor" id="Daxmeasures"></a>

1. Total Subscribers (M) =<br>
DIVIDE(SUM(view_1_Top_100_youtube_channels[total_subscribers]),1000000)

3. Total Views (B) =<br>
DIVIDE(SUM(view_1_Top_100_youtube_channels[total_views]),1000000000)

6. Total Videos =<br>
SUM(view_1_Top_100_youtube_channels[total_videos])

8. Average Views Per Video (M) =<br>
DIVIDE(SUM(view_1_Top_100_youtube_channels[total_views]),SUM(view_1_Top_100_youtube_channels[total_videos]))

10. Subscriber Engagement Rate =<br>
DIVIDE(SUM(view_1_Top_100_youtube_channels[total_subscribers]),SUM(view_1_Top_100_youtube_channels[total_videos]))

12. Views per subscriber =<br>
DIVIDE(SUM(view_1_Top_100_youtube_channels[total_views]),SUM(view_1_Top_100_youtube_channels[total_subscribers]))
<br>

### Analysis <a class="anchor" id="Analysis"></a>

1. Who are the top 3 YouTubers with the most subscribers?

| Rank | Channel Name |	Subscribers (M) |
| --- | ------------- | --------------- |
| 1 | NoCopyrightSounds | 33.60 |
| 2 | DanTDM | 28.60 |
| 3 |	Dan Rhodes |	26.50 |

<br>

2. Which 3 channels have uploaded the most videos?

| Rank |	Channel Name |	Videos Uploaded |
| --- | ---------------- | ---------------- |
| 1 | 24 News HD |	165,103 |
| 2 |	Sky News |	46,009 |
|3 |	BBC News |	40,179 |

<br>

3. Which 3 channels have the most views?

| Rank |	Channel Name |	Total Views (B) |
| --- | ---------------- | ---------------- |
| 1 |	DanTDM |	19.78 |
| 2 |	Dan Rhodes |	18.56 |
| 3 |	Mister Max |	15.97 |

<br>

4. Which 3 channels have the highest average views per video?

| Rank | Channel Name |	Averge Views per Video (M) |
| ---- | ------------ | -------------------------- |
| 1 | Mark Ronson |	322.7 |
| 2 | Jessie J |	59.7 |
| 3 | Dua Lipa |	57.6 |

<br>

5. What is the average view per video of the top 3 YouTubers with most subscriber count?

| Rank | Channel Name |	Averge Views per Video (M) |
| ---- | ------------ | -------------------------- |
| 1 | NoCopyrightSounds | 6.92 |
| 2 | DanTDM | 5.34 |
| 3 |	Dan Rhodes | 11.15 |

<br>

### Discovery <a class="anchor" id="Discovery"></a>

●  "NoCopyrightSOunds", "DanTDM" and "Dan Rhodes" are the YouTubers with the most subscribers in the UK. <br>
●  "Mark Ronson", "Jessie J" and "Dua Lipa" are the YouTubers with the highest average view per video. <br>
●  "DanTDM", "Dan RHodes" and "Mister Max" are the YouTubers with the most views. <br>
●  "24 News HD", "Sky News" and "BBC News" are the channels with the most videos uploaded.

<br>

### Recommendation <a class="anchor" id="Recommendation"></a>


    1. Dan Rhodes is the best YouTube channel to collaborate with to maximize visbility because this channel has the highest average view per video among the top 3 YouTubers with the most subscribers count in the UK.
    
    2. The other 2 channels to form collaborations with are NoCopyrightSounds and DanTDM based on this analysis, because they attract the most engagement on their channels consistently from their large amount of subscribers.
    
    3. Mark Ronson, Jessie J and Dua Lipa are the top 3 YouTubers with the highest average view per video, although their subscriber count is far less than the channels mentioned earlier, and collaborating with them has good potential return on investments.
    
    4. Although 24 News HD, Sky News and BBC News are regular publishers on YouTube, it is important to carefully analyze whether collaborating with them is worth the effort, as the potential return on investments is significantly lower compared to the other channels.     